In [2]:
#16.6.1

import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version 

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,937 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages 

In [3]:
#Start Spark Session

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tokens").getOrCreate()

In [4]:
#Import Tokenizer library
from pyspark.ml.feature import Tokenizer

In [5]:
#Create a sample Data Frame
dataFrame = spark.createDataFrame([
    (0, "Spark is great"),
    (1, "We are learning Spark"),
    (2, "Spark is better than Hadoop no doubt"),
    (3, "This is first time for me using Spark")
], ["ID","Sentence"])

dataFrame.show()

+---+--------------------+
| ID|            Sentence|
+---+--------------------+
|  0|      Spark is great|
|  1|We are learning S...|
|  2|Spark is better t...|
|  3|This is first tim...|
+---+--------------------+



In [6]:
#Tokenize sentences 

tokenizer = Tokenizer(inputCol="Sentence", outputCol="Words")


In [7]:
#Tranform and Show Data Frame

tokenized_df= tokenizer.transform(dataFrame)
tokenized_df.show(truncate=False)

+---+-------------------------------------+----------------------------------------------+
|ID |Sentence                             |Words                                         |
+---+-------------------------------------+----------------------------------------------+
|0  |Spark is great                       |[spark, is, great]                            |
|1  |We are learning Spark                |[we, are, learning, spark]                    |
|2  |Spark is better than Hadoop no doubt |[spark, is, better, than, hadoop, no, doubt]  |
|3  |This is first time for me using Spark|[this, is, first, time, for, me, using, spark]|
+---+-------------------------------------+----------------------------------------------+



In [8]:
# Create a function to return the length of a list
def word_list_length(word_list):
    return len(word_list)

In [9]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [10]:
# Create a user defined function

count_tokens = udf(word_list_length, IntegerType())

In [11]:
#Select the needed columns and dont truncate results

tokenized_df.withColumn("Tokens", count_tokens(col("Words"))).show(truncate= False)

+---+-------------------------------------+----------------------------------------------+------+
|ID |Sentence                             |Words                                         |Tokens|
+---+-------------------------------------+----------------------------------------------+------+
|0  |Spark is great                       |[spark, is, great]                            |3     |
|1  |We are learning Spark                |[we, are, learning, spark]                    |4     |
|2  |Spark is better than Hadoop no doubt |[spark, is, better, than, hadoop, no, doubt]  |7     |
|3  |This is first time for me using Spark|[this, is, first, time, for, me, using, spark]|8     |
+---+-------------------------------------+----------------------------------------------+------+



#16.6.2 Stop Words

In [21]:
#Create a Data Frame

sentencedf = spark.createDataFrame([
    (0, ["Big", "data", "is","super","powerful"]),
    (1, ["This","is","going","to","be","epic"])
],["ID","Raw"])

sentencedf.show(truncate=False)

+---+--------------------------------+
|ID |Raw                             |
+---+--------------------------------+
|0  |[Big, data, is, super, powerful]|
|1  |[This, is, going, to, be, epic] |
+---+--------------------------------+



In [22]:
# Import stop words library

from pyspark.ml.feature import StopWordsRemover

In [23]:
#Run The Remover

remover = StopWordsRemover(inputCol="Raw", outputCol="Filtered")

In [24]:
#Transform and Show data
remover.transform(sentencedf).show(truncate=False)

+---+--------------------------------+----------------------------+
|ID |Raw                             |Filtered                    |
+---+--------------------------------+----------------------------+
|0  |[Big, data, is, super, powerful]|[Big, data, super, powerful]|
|1  |[This, is, going, to, be, epic] |[going, epic]               |
+---+--------------------------------+----------------------------+



16.6.3

In [25]:
#Import

from pyspark.ml.feature import HashingTF, IDF

In [28]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_16/airlines.csv"
spark.sparkContext.addFile(url)
airline_df = spark.read.csv(SparkFiles.get("airlines.csv"), sep=",", header=True)

# Show DataFrame
airline_df.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------+
|Airline Tweets                                                                                                                         |
+---------------------------------------------------------------------------------------------------------------------------------------+
|@VirginAmerica plus you've added commercials to the experience... tacky.                                                               |
|@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing. it's really the only bad thing about flying VA|
|@VirginAmerica do you miss me? Don't worry we'll be together very soon.                                                                |
|@VirginAmerica Are the hours of operation for the Club at SFO that are posted online current?                                          |
|@VirginAmerica awaiting my return

In [29]:
#Tokenize Data Frame

tok = Tokenizer(inputCol="Airline Tweets", outputCol="Words")
tok_transformed= tok.transform(airline_df)
tok_transformed.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Airline Tweets                                                                                                                         |Words                                                                                                                                                          |
+---------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|@VirginAmerica plus you've added commercials to the experience... tacky.                                 

In [34]:
# Remove the Stop Words

removed = StopWordsRemover(inputCol="Words", outputCol="Filtered")
removed_transform = removed.transform(tok_transformed)
removed_transform.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------+
|Airline Tweets                                                                                                                         |Words                                                                                                                                                          |Filtered                                                                                       |
+---------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------

In [36]:
#Run the Hashing term frequency
hasing = HashingTF(inputCol="Filtered", outputCol="Hashed Values", numFeatures=pow(2,18))

#Transform into DF
hashed_df= hasing.transform(removed_transform)
hashed_df.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+
|Airline Tweets                                                                                                                         |Words                                                                                                                                                          |Filtered                                                                                       |Hashed Values                                                            

In [40]:
#Fit the IDF on the Data Set 
idf = IDF(inputCol="Hashed Values", outputCol="Features")
idfModel= idf.fit(hashed_df)
rescaledData= idfModel.transform(hashed_df)

#Display the Data Frame
rescaledData.show()

#Display The Data Frame

rescaledData.select("Words", "Features").show(truncate=False)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|      Airline Tweets|               Words|            Filtered|       Hashed Values|            Features|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|@VirginAmerica pl...|[@virginamerica, ...|[@virginamerica, ...|(262144,[1419,999...|(262144,[1419,999...|
|@VirginAmerica se...|[@virginamerica, ...|[@virginamerica, ...|(262144,[30053,44...|(262144,[30053,44...|
|@VirginAmerica do...|[@virginamerica, ...|[@virginamerica, ...|(262144,[107065,1...|(262144,[107065,1...|
|@VirginAmerica Ar...|[@virginamerica, ...|[@virginamerica, ...|(262144,[9641,506...|(262144,[9641,506...|
|@VirginAmerica aw...|[@virginamerica, ...|[@virginamerica, ...|(262144,[6122,505...|(262144,[6122,505...|
+--------------------+--------------------+--------------------+--------------------+--------------------+

+-----------------------------------

#16.6.4